In [1]:
import xarray as xr
import pandas as pd
import geopandas as gpd
import pendulum    
from dateutil.relativedelta import relativedelta
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from skill import config
from skill import recorte

In [2]:
dir_dados = config.dir_dados
dir_shp = config.dir_shp
dir_img = config.dir_img

In [3]:
meses=['', 'Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

Função para criar um dataframe vazio com os meses nas linhas e colunas.

In [4]:
def df_corr():
        
        df_completo = pd.DataFrame(
                index=['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro'],
        columns=['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro'],
        )
        
        return df_completo

In [5]:
dir_dados.joinpath('table_corr').mkdir(exist_ok=True)
dir_img.joinpath('table_corr', 'cam3').mkdir(exist_ok=True)
dir_img.joinpath('table_corr', 'cfsv2').mkdir(exist_ok=True)

# Criando tabelas para dados do cam

In [12]:
for shapefile in ['Sao_Francisco.shp', 'Grande.shp', 'Parana.shp', 'Xingu.shp', 'Iguacu.shp', 'Tocantins.shp', 'Paranapanema.shp']:

    shp = gpd.read_file(dir_shp.joinpath(shapefile))
    df_completo = df_corr()

    for i in range(1,13):

        date = pendulum.now().set(year=1975, month=i, day=21)

        ds = xr.open_dataset(dir_dados.joinpath(f'cam3-anom_corr-init{i}.nc'))

        ds_recortado = recorte.recortar(ds, shp)
        ds_media = ds_recortado.mean(dim=['lat', 'lon'])

        df = ds_media.to_dataframe().drop('spatial_ref', axis=1)

        mes1 = (date + relativedelta(months=1)).month
        mes2 = (date + relativedelta(months=2)).month
        mes3 = (date + relativedelta(months=3)).month

        index = [meses[mes1], meses[mes2], meses[mes3]]
        df.index = index

        df_completo[meses[i]] = df

    df_completo.T.to_csv(dir_dados.joinpath('table_corr', f'cam3-{shp.Nome_Bacia.item()}.csv'))

    fig, ax = plt.subplots(figsize=(20,10))

    sns.heatmap(df_completo.T, vmin=-1, vmax=1, cmap='RdBu', annot=True, ax=ax)
    ax.set_xlabel('Mês de Previsão', fontsize=14)
    ax.set_ylabel('Mês da Condição Inicial', fontsize=14)
    ax.set_title(shp.Nome_Bacia.item(), fontsize=16)

    plt.close('all')

    fig.savefig(dir_img.joinpath('table_corr', 'cam3', f'cam3-{shp.Nome_Bacia.item()}.png'), bbox_inches='tight', facecolor='w', edgecolor='w')

# Criando tabelas para dados do CFSv2

In [6]:
for shapefile in ['Sao_Francisco.shp', 'Grande.shp', 'Parana.shp', 'Xingu.shp', 'Iguacu.shp', 'Tocantins.shp', 'Paranapanema.shp']:
    
    shp = gpd.read_file(dir_shp.joinpath(shapefile))
    df_completo = df_corr()

    for i in range(1,13):
        
        date = pendulum.now().set(year=1975, month=i, day=21)

        ds = xr.open_dataset(dir_dados.joinpath('outros_centros', f"{date.format('MMM')}IC_cfsv2_precip_skill.nc"))[['sk1', 'sk2', 'sk3']]

        ds_recortado = recorte.recortar(ds, shp)
        ds_media = ds_recortado.mean(dim=['lat', 'lon'])

        df = ds_media.to_dataframe().drop('spatial_ref', axis=1).T

        mes1 = (date + relativedelta(months=1)).month
        mes2 = (date + relativedelta(months=2)).month
        mes3 = (date + relativedelta(months=3)).month

        index = [meses[mes1], meses[mes2], meses[mes3]]
        df.index = index

        df_completo[meses[i]] = df
    
    df_completo.T.to_csv(dir_dados.joinpath('table_corr', f'cfsv2-{shp.Nome_Bacia.item()}.csv'))

    fig, ax = plt.subplots(figsize=(20,10))

    sns.heatmap(df_completo.T, vmin=-1, vmax=1, cmap='RdBu', annot=True, ax=ax)
    ax.set_xlabel('Mês de Previsão', fontsize=14)
    ax.set_ylabel('Mês da Condição Inicial', fontsize=14)
    ax.set_title(shp.Nome_Bacia.item(), fontsize=16)

    plt.close('all')

    fig.savefig(dir_img.joinpath('table_corr', 'cfsv2', f'cfsv2-{shp.Nome_Bacia.item()}.png'), bbox_inches='tight', facecolor='w', edgecolor='w')